# Data Preparation

Let's load the data into a h5ad object. Dataset is available at [PRIDE](https://www.ebi.ac.uk/pride/archive/projects/PXD015912).

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import torch

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

from scp.models import PROTVI
import scp.plots as pl
import scp.utils as utils
import scp.benchmark_models as bm

import scvi

np.random.seed(0)
scvi.settings.seed = 0

Global seed set to 0


Let's define the path to the data and load it into anndata.

In [2]:
DATA_DIR = "../../data/raw/poulos2020"
DATA_PATH = os.path.join(DATA_DIR, "Peptide_intensity_matrix_b9369842-f3cf-4383-9e5c-6734dadcfbc9.csv")
ANNOTATION_PATH = os.path.join(DATA_DIR, "Mapping_file_PXD015912.xlsx")
GROUPING_PATH = os.path.join(DATA_DIR, "pseudo_groups.xlsx")

In [3]:
data = pd.read_csv(DATA_PATH, sep="\t")
annotations = pd.read_excel(ANNOTATION_PATH, sheet_name="PXD015912")
groups = pd.read_excel(GROUPING_PATH, sheet_name="Supplementary_Data_2", header=2)

In [4]:
var_cols = [c for c in data.columns if not c[0].isdigit()]
vars = data[var_cols]

data.drop(var_cols, axis=1, inplace=True)

Let's log normalize.

In [5]:
data = data.T
data = np.log(data + 1)

In [6]:
obs = annotations.copy()
obs = pd.merge(obs, groups, left_on="Filename", right_on="File Name")

obs["name"] = obs["Filename"].str.lower()

In [7]:
obs = pd.merge(obs, pd.Series(data.index, name="name"), left_on="name", right_on="name")
obs.set_index("name", inplace=True)

data = data.loc[obs.index]

In [8]:
obs.sort_index(inplace=True)
data.sort_index(inplace=True)

In [9]:
info = obs["Information"].str.split("_")

obs["day"] = [i[0] for i in info]
obs["method"] = [i[1] for i in info]
obs["sample"] = [i[2].split(".")[0] for i in info]

obs["ratio"] = [("_".join(i[2].split(".")[1:]))[1:] for i in info]

In [10]:
def get_ratio(raw):
    raw = raw.rstrip()
    raw = raw.lstrip()
    ratio = raw.split("%")[0]
    ratio = ratio.replace("_", ".")
    fratio = float(ratio) / 100
    return fratio

obs["ovary"] = 0.0
obs["prostate"] = 0.0
obs["yeast"] = 0.0
obs["hek293t"] = 0.0

for i, row in obs.iterrows():
    ratio = row["ratio"]
    if ratio.startswith("HEK293T"):
        obs.loc[i, "hek293t"] = 1
    else:
        types = ratio.split("/")

        assert len(types) == 3

        assert "Ovary" in types[0]
        obs.loc[i, "ovary"] = get_ratio(types[0])
        
        assert "Prostate" in types[1]
        obs.loc[i, "prostate"] = get_ratio(types[1])

        assert "Yeast" in types[2]
        obs.loc[i, "yeast"] = get_ratio(types[2])

In [11]:
obs["control"] = (obs["hek293t"] == 1).astype(int)
obs["ovary"] = obs["ovary"].astype("category")
obs["prostate"] = obs["prostate"].astype("category")
obs["yeast"] = obs["yeast"].astype("category")

In [12]:
adata = sc.AnnData(X=data, obs=obs, var=vars)

adata = adata[~np.all(np.isnan(adata.X), axis=1), :].copy()
adata = adata[:, ~np.all(np.isnan(adata.X), axis=0)].copy()

adata.layers["raw"] = adata.X.copy()

adata.obs["sample_no"] = adata.obs["sample"].astype("category").cat.codes
adata.var["species"] = adata.var["Protein"].str.split("_", expand=True)[1]

adata.var["Peptide"] = adata.var["Peptide"].astype(str)

In [14]:
adata.obs["day"] = adata.obs["day"].astype("category")
adata.obs["method"] = adata.obs["method"].astype("category")
adata.obs["sample"] = adata.obs["sample"].astype("category")
adata.obs["ratio"] = adata.obs["ratio"].astype("category")
adata.obs["sample_no"] = adata.obs["sample_no"].astype("category")

In [15]:
adata

AnnData object with n_obs × n_vars = 1527 × 18114
    obs: 'Filename', 'Filetype', 'Information', 'File Name', 'Date', 'Sample', 'Instrument', 'Replicate', 'Day', 'PseudoReplicate_2', 'PseudoReplicate_3', 'PseudoReplicate_4', 'PseudoReplicate_5', 'PseudoReplicate_6', 'day', 'method', 'sample', 'ratio', 'ovary', 'prostate', 'yeast', 'hek293t', 'control', 'sample_no'
    var: 'Protein', 'Peptide', 'ModifiedPeptide', 'species'
    layers: 'raw'

In [16]:
adata.write("../../data/processed/poulos2020.h5ad")